In [1]:
import numpy as np
import pandas as pd
from astropy.io import ascii
from astropy.table import Table
from tqdm import tqdm

In [2]:
useful_cols = [
    "id",
    "best.universe.redshift",
    "bayes.agn.fracAGN",
    "best.agn.fracAGN",
    "bayes.agn.fracAGN_err",
    "bayes.agn.luminosity",
    "best.agn.luminosity",
    "bayes.agn.total_dust_luminosity",
    "best.agn.total_dust_luminosity",
    "bayes.dust.luminosity",
    "bayes.dust.luminosity_err",
    "best.dust.luminosity",
    "best.reduced_chi_square",
    "bayes.stellar.m_star",
    "bayes.stellar.m_star_err",
    "best.stellar.m_star",
    "bayes.sfh.sfr",
    "bayes.sfh.sfr_err",
    "best.sfh.sfr",
    "best.universe.luminosity_distance",
    "bayes.sfh.age",
    "best.sfh.age"
]

# 3-30

## Paths

In [3]:
cat_dirSK = "/net/virgo01/data/users/ebbers/BSc_Thesis/SKIRTOR/3-30SK/"
cat_dirF = "/net/virgo01/data/users/ebbers/BSc_Thesis/Fritz/3-30F/"

COSMOS_dirSK = "/net/virgo01/data/users/ebbers/BSc_Thesis/COSMOS/SKIRTOR/SK3-30/"
COSMOS_dirF= "/net/virgo01/data/users/ebbers/BSc_Thesis/COSMOS/Fritz/F3-30/"

sed_dirSK = cat_dirSK
sed_dirF = cat_dirF

cat_nameSK = "results_3-30SK.fits"
cat_nameF = "results_3-30F.fits"
#Ant_COSMOS_SK = "CIGALE_COSMOS2020_XRAY_FIR_COSMOS-Web_z_mass_chi_sel_cutouts.fits" #Antonio's results
COSMOS_nameSK = 'results_SK3-30.fits'
COSMOS_nameF = 'results_COSMOSF3-30.fits'

observations_fileSK = cat_dirSK + "observations_3-30SK.fits"
observations_fileF = cat_dirF + "observations_3-30F.fits"
COSMOS_observations_fileF = COSMOS_dirF + 'observations_COSMOSF3-30.fits'
COSMOS_observations_fileSK = COSMOS_dirSK + 'observations_SK3-30.fits'

In [4]:
resultsSK = Table.read(cat_dirSK + cat_nameSK, format="fits").to_pandas()
resultsF = Table.read(cat_dirF + cat_nameF, format="fits").to_pandas()
COSMOSF = Table.read(COSMOS_dirF + COSMOS_nameF, format="fits").to_pandas()
COSMOSSK = Table.read(COSMOS_dirSK + COSMOS_nameSK, format="fits").to_pandas()
#dfANT = Table.read(COSMOS_dirSK + Ant_COSMOS_SK, format="fits").to_pandas() #does not need to be merged with observation file

In [5]:
observationsSK = Table.read(observations_fileSK, format="fits").to_pandas()

dfSK = pd.merge(
    resultsSK[useful_cols],
    observationsSK,
    on="id",
)

observationsF = Table.read(observations_fileF, format="fits").to_pandas()

dfF = pd.merge(
    resultsF[useful_cols],
    observationsF,
    on="id",
)

observations_COSMOSF = Table.read(COSMOS_observations_fileF, format = "fits").to_pandas()

dfCOSMOSF = pd.merge(
    COSMOSF[useful_cols],
    observations_COSMOSF,
    on="id",
)

observations_COSMOSSK = Table.read(COSMOS_observations_fileSK, format = "fits").to_pandas()

dfCOSMOSSK = pd.merge(
    COSMOSSK[useful_cols],
    observations_COSMOSSK,
    on="id",
)

## Read catalogue and select possible good candidates

Additional criteria:

- $\frac{1}{5} \leq \frac{M_{*, best}}{M_{*, bayes}} \leq 5$

- $\frac{1}{5} \leq \frac{SFR_{best}}{SFR_{bayes}} \leq 5$

- $\frac{1}{5} \leq \frac{f_{AGN, best}}{f_{AGN, bayes}} \leq 5$

- $\frac{f_{AGN,\ bayes}}{f_{AGN, bayes\_err}} > 1$

- reduced $\chi^2 < 5$

In [12]:
mass_ratio = dfSK["best.stellar.m_star"] / dfSK["bayes.stellar.m_star"]
sfr_ratio = dfSK["best.sfh.sfr"] / dfSK["bayes.sfh.sfr"]
fagn_ratio = dfSK["best.agn.fracAGN"] / dfSK["bayes.agn.fracAGN"]
SN_ratio = dfSK["bayes.agn.fracAGN"]/ dfSK["bayes.agn.fracAGN_err"]

mass_sel = (mass_ratio >= 1 / 5) & (mass_ratio <= 5)
sfr_sel = (sfr_ratio >= 1 / 5) & (sfr_ratio <= 5)
fagn_sel = (fagn_ratio >= 1 / 5) & (fagn_ratio <= 5)
SN_sel = (SN_ratio > 1)

chi_sel = dfSK["best.reduced_chi_square"] < 5
selection = mass_sel & sfr_sel & fagn_sel & chi_sel & SN_sel

df_SK = dfSK[selection]
df_SK = df_SK.sort_values("id")

df_SK

,id,best.universe.redshift,bayes.agn.fracAGN,best.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.luminosity,best.agn.luminosity,bayes.agn.total_dust_luminosity,best.agn.total_dust_luminosity,bayes.dust.luminosity,...,herschel.pacs.160,herschel.pacs.160_err,PSW,PSW_err,PMW,PMW_err,PLW,PLW_err,eROSITA_soft,eROSITA_soft_err
388,39890303330569440,0.590000,0.634043,0.55,0.143080,7.777672e+37,8.461149e+37,7.737812e+37,8.424944e+37,1.528154e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19926,39890307625535289,0.690000,0.613012,0.70,0.150191,1.018874e+38,1.275749e+38,1.014242e+38,1.270290e+38,1.959718e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1840,39890316215468063,0.560000,0.342546,0.85,0.286590,1.538263e+37,5.288356e+37,1.516087e+37,5.265727e+37,6.731804e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6300,39890316215471241,0.509000,0.386666,0.40,0.245802,1.013124e+37,4.844896e+36,1.000598e+37,4.805192e+36,3.557505e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000002,6.698264e-07
15634,39890445064488671,0.620000,0.491138,0.70,0.242989,2.895719e+37,3.452277e+37,2.872669e+37,3.423985e+37,8.377925e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30148,43132758825921346,0.389340,0.199849,0.20,0.173227,1.567929e+37,1.784382e+37,1.556764e+37,1.776747e+37,2.194846e+38,...,53.405205,48.241022,43.695751,8.604984,24.256,8.42665,NaN,NaN,NaN,NaN
14748,43132763120886340,0.680000,0.245527,0.30,0.231704,1.652775e+37,1.901976e+37,1.640413e+37,1.886389e+37,1.329315e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13525,43132763120886461,0.620000,0.247627,0.70,0.239477,1.225920e+37,3.742422e+37,1.216793e+37,3.726409e+37,9.894647e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10126,43132771710821413,0.595074,0.381504,0.55,0.197629,1.751312e+38,3.792095e+38,1.639258e+38,3.716925e+38,6.820711e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000008,1.413871e-06


In [13]:
mass_ratio = dfF["best.stellar.m_star"] / dfF["bayes.stellar.m_star"]
sfr_ratio = dfF["best.sfh.sfr"] / dfF["bayes.sfh.sfr"]
fagn_ratio = dfF["best.agn.fracAGN"] / dfF["bayes.agn.fracAGN"]
SN_ratio = dfF["bayes.agn.fracAGN"]/ dfF["bayes.agn.fracAGN_err"]

mass_sel = (mass_ratio >= 1 / 5) & (mass_ratio <= 5)
sfr_sel = (sfr_ratio >= 1 / 5) & (sfr_ratio <= 5)
fagn_sel = (fagn_ratio >= 1 / 5) & (fagn_ratio <= 5)
SN_sel = (SN_ratio > 1)

chi_sel = dfF["best.reduced_chi_square"] < 5
selection = mass_sel & sfr_sel & fagn_sel & chi_sel & SN_sel

df_F = dfF[selection]

df_F = df_F.sort_values("id")
df_F

,id,best.universe.redshift,bayes.agn.fracAGN,best.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.luminosity,best.agn.luminosity,bayes.agn.total_dust_luminosity,best.agn.total_dust_luminosity,bayes.dust.luminosity,...,herschel.pacs.160,herschel.pacs.160_err,PSW,PSW_err,PMW,PMW_err,PLW,PLW_err,eROSITA_soft,eROSITA_soft_err
388,39890303330569440,0.590000,0.790273,0.70,0.110102,2.613719e+38,2.013204e+38,2.562603e+38,1.963400e+38,1.649729e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1535,39890307625533645,0.330000,0.323506,0.30,0.316410,4.349322e+37,1.893528e+36,4.302900e+37,1.793575e+36,1.854292e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19926,39890307625535289,0.690000,0.841585,0.85,0.041531,3.910921e+38,3.468913e+38,3.830807e+38,3.383096e+38,1.794281e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24317,39890307625539408,0.650000,0.312093,0.40,0.302126,1.731745e+38,2.051359e+37,1.713102e+38,2.000611e+37,9.877273e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1840,39890316215468063,0.560000,0.444256,0.85,0.307011,3.924428e+37,4.830228e+37,3.832537e+37,4.575256e+37,6.976843e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10126,43132771710821413,0.595074,0.339267,0.70,0.191598,1.439020e+38,8.138549e+38,1.298167e+38,7.708942e+38,5.907594e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000008,0.000001
42109,43132771710826283,0.580000,0.478196,0.85,0.299451,9.931284e+37,3.414564e+38,9.794533e+37,3.383574e+38,1.418599e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27217,43132776005782281,0.290000,0.449342,0.70,0.277079,9.762442e+36,1.320792e+37,9.564747e+36,1.288118e+37,1.621143e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24348,43132776005785947,0.630000,0.322266,0.20,0.304922,1.869386e+39,1.006595e+38,1.848765e+39,9.534597e+37,1.163377e+39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
mass_ratio = dfCOSMOSF["best.stellar.m_star"] / dfCOSMOSF["bayes.stellar.m_star"]
sfr_ratio = dfCOSMOSF["best.sfh.sfr"] / dfCOSMOSF["bayes.sfh.sfr"]
fagn_ratio = dfCOSMOSF["best.agn.fracAGN"] / dfCOSMOSF["bayes.agn.fracAGN"]
SN_ratio = dfCOSMOSF["bayes.agn.fracAGN"]/ dfCOSMOSF["bayes.agn.fracAGN_err"]

mass_sel = (mass_ratio >= 1 / 5) & (mass_ratio <= 5)
sfr_sel = (sfr_ratio >= 1 / 5) & (sfr_ratio <= 5)
fagn_sel = (fagn_ratio >= 1 / 5) & (fagn_ratio <= 5)
SN_sel = (SN_ratio > 1)

chi_sel = dfCOSMOSF["best.reduced_chi_square"] < 5
selection = mass_sel & sfr_sel & fagn_sel & chi_sel & SN_sel

df_COSMOSF = dfCOSMOSF[selection]

df_COSMOSF = df_COSMOSF.sort_values("id")
df_COSMOSF

,id,best.universe.redshift,bayes.agn.fracAGN,best.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.luminosity,best.agn.luminosity,bayes.agn.total_dust_luminosity,best.agn.total_dust_luminosity,bayes.dust.luminosity,...,subaru.suprime.IA767,subaru.suprime.IA767_err,subaru.suprime.IB827,subaru.suprime.IB827_err,subaru.suprime.NB711,subaru.suprime.NB711_err,subaru.suprime.NB816,subaru.suprime.NB816_err,vista.vircam.NB118,vista.vircam.NB118_err
62845,274.0,2.1539,0.566878,0.55,0.108525,4.495974e+37,3.239056e+37,1.283757e+37,1.026534e+37,4.243759e+37,...,0.001209,0.000123,0.001177,0.000121,0.001395,0.000142,0.001070,0.000108,0.001810,0.000187
25955,670.0,1.1792,0.320331,0.30,0.058782,1.634193e+37,1.471004e+37,4.562877e+36,4.078075e+36,4.108211e+37,...,0.002830,0.000285,0.004133,0.000415,0.002753,0.000278,0.004291,0.000430,0.005639,0.000566
25956,701.0,1.1539,0.295364,0.85,0.276163,4.599400e+36,2.889215e+37,4.527824e+36,2.862993e+37,1.113806e+37,...,0.000930,0.000099,0.001316,0.000137,0.000851,0.000096,0.001418,0.000143,0.001954,0.000204
25957,707.0,1.1951,0.375625,0.40,0.342221,5.728223e+36,3.630764e+35,5.593574e+36,9.284817e+34,1.323610e+36,...,0.000110,0.000027,0.000272,0.000041,0.000078,0.000028,0.000218,0.000026,0.000177,0.000044
12781,942.0,0.9000,0.408615,0.99,0.354871,2.320523e+37,1.808570e+38,2.297279e+37,1.792156e+38,4.816873e+36,...,0.000737,0.000078,0.000860,0.000090,0.000737,0.000080,0.000978,0.000101,0.001403,0.000146
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72688,964385.0,2.4158,0.428911,0.99,0.348010,6.050087e+37,4.582829e+38,5.852499e+37,4.340917e+38,1.198339e+37,...,0.000111,0.000022,0.000092,0.000022,0.000031,0.000020,0.000119,0.000019,NaN,NaN
47964,964386.0,1.3068,0.557427,0.85,0.264928,1.814004e+37,1.786721e+37,1.752325e+37,1.742519e+37,6.872100e+36,...,0.000255,0.000032,0.000196,0.000028,0.000288,0.000036,0.000266,0.000030,NaN,NaN
80474,964391.0,2.8983,0.377935,0.20,0.318105,4.085321e+37,2.968771e+36,3.933048e+37,7.591927e+35,1.090375e+37,...,0.000006,0.000019,0.000030,0.000020,0.000075,0.000021,0.000040,0.000015,NaN,NaN
47967,964428.0,1.2587,0.544781,0.70,0.240343,4.161506e+37,2.989148e+37,4.055912e+37,2.915200e+37,3.743137e+37,...,0.002323,0.000237,0.002577,0.000259,NaN,NaN,NaN,NaN,0.007177,0.000719


In [23]:
mass_ratio = dfCOSMOSSK["best.stellar.m_star"] / dfCOSMOSSK["bayes.stellar.m_star"]
sfr_ratio = dfCOSMOSSK["best.sfh.sfr"] / dfCOSMOSSK["bayes.sfh.sfr"]
fagn_ratio = dfCOSMOSSK["best.agn.fracAGN"] / dfCOSMOSSK["bayes.agn.fracAGN"]
SN_ratio = dfCOSMOSSK["bayes.agn.fracAGN"]/ dfCOSMOSSK["bayes.agn.fracAGN_err"]

mass_sel = (mass_ratio >= 1 / 5) & (mass_ratio <= 5)
sfr_sel = (sfr_ratio >= 1 / 5) & (sfr_ratio <= 5)
fagn_sel = (fagn_ratio >= 1 / 5) & (fagn_ratio <= 5)
SN_sel = (SN_ratio > 1)

chi_sel = dfCOSMOSSK["best.reduced_chi_square"] < 5
selection = mass_sel & sfr_sel & fagn_sel & chi_sel & SN_sel

df_COSMOSSK = dfCOSMOSSK[selection]

df_COSMOSSK = df_COSMOSSK.sort_values("id")
df_COSMOSSK

,id,best.universe.redshift,bayes.agn.fracAGN,best.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.luminosity,best.agn.luminosity,bayes.agn.total_dust_luminosity,best.agn.total_dust_luminosity,bayes.dust.luminosity,...,subaru.suprime.IA767,subaru.suprime.IA767_err,subaru.suprime.IB827,subaru.suprime.IB827_err,subaru.suprime.NB711,subaru.suprime.NB711_err,subaru.suprime.NB816,subaru.suprime.NB816_err,vista.vircam.NB118,vista.vircam.NB118_err
62845,274.0,2.1539,0.661199,0.70,0.094964,4.814398e+37,4.823257e+37,2.513474e+37,4.727647e+37,5.018624e+37,...,0.001209,0.000123,0.001177,0.000121,0.001395,0.000142,0.001070,0.000108,0.001810,0.000187
25955,670.0,1.1792,0.435028,0.40,0.102657,1.942275e+37,1.964567e+37,6.362243e+36,5.950924e+36,3.668468e+37,...,0.002830,0.000285,0.004133,0.000415,0.002753,0.000278,0.004291,0.000430,0.005639,0.000566
62847,947.0,2.0474,0.702788,0.70,0.026138,2.365818e+38,2.396660e+38,2.355536e+38,2.386404e+38,3.340786e+38,...,0.000253,0.000038,0.000442,0.000056,0.000121,0.000039,0.000312,0.000036,0.002052,0.000211
62848,1063.0,2.4081,0.401932,0.99,0.370928,1.963825e+36,1.135488e+37,1.910143e+36,1.126183e+37,4.611303e+35,...,0.000031,0.000022,NaN,NaN,0.000066,0.000029,0.000057,0.000015,0.000032,0.000038
47968,1068.0,1.6465,0.863899,0.85,0.047240,2.577167e+37,1.761127e+37,2.542533e+37,1.726217e+37,1.243632e+37,...,0.000098,0.000024,0.000101,0.000026,0.000133,0.000030,0.000107,0.000019,0.000295,0.000045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72688,964385.0,2.4158,0.526794,0.99,0.377385,6.545913e+37,4.151275e+38,6.465597e+37,4.133512e+38,1.101851e+37,...,0.000111,0.000022,0.000092,0.000022,0.000031,0.000020,0.000119,0.000019,NaN,NaN
47964,964386.0,1.3068,0.589256,0.70,0.208200,3.441849e+36,2.781638e+36,3.221754e+36,2.726498e+36,6.335196e+36,...,0.000255,0.000032,0.000196,0.000028,0.000288,0.000036,0.000266,0.000030,NaN,NaN
80474,964391.0,2.8983,0.341509,0.30,0.283239,1.289833e+37,3.498150e+36,1.221503e+37,1.226156e+36,1.075698e+37,...,0.000006,0.000019,0.000030,0.000020,0.000075,0.000021,0.000040,0.000015,NaN,NaN
47967,964428.0,1.2587,0.382373,0.55,0.157672,7.684803e+36,1.262061e+37,6.780734e+36,1.256661e+37,3.562242e+37,...,0.002323,0.000237,0.002577,0.000259,NaN,NaN,NaN,NaN,0.007177,0.000719


In [17]:
IRlum_SK = pd.DataFrame(columns = ["id", "redshift", "IRLum_tot", "fAGN_IR","fAGN_err", 'best_fAGN'])
IR_lum = []

z = df_SK["best.universe.redshift"]
fAGN = df_SK["bayes.agn.fracAGN"]
fAGN_err = df_SK["bayes.agn.fracAGN_err"]
best_fAGN = df_SK["best.agn.fracAGN"]

for i in tqdm(df_SK.index):
    
    sed = Table.read(cat_dirSK + str(df_SK["id"][i]) + "_best_model.fits", format="fits").to_pandas()
    w_spec = np.array(sed["wavelength"])
    
    three = np.where(3000 <= sed["wavelength"])[0][0]
    thir = np.where(30000 <= sed["wavelength"])[0][0]
    
    IR_lum.append(np.trapz(sed["L_lambda_total"][three:thir], w_spec[three:thir]))
    

IRlum_SK = pd.concat([IRlum_SK, pd.DataFrame(np.column_stack([df_SK["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_SK.columns)], ignore_index=True)

100%|██████████| 6744/6744 [06:53<00:00, 16.32it/s]
/var/lib/condor/execute/dir_335412/ipykernel_335502/2053908816.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IRlum_SK = pd.concat([IRlum_SK, pd.DataFrame(np.column_stack([df_SK["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_SK.columns)], ignore_index=True)


In [18]:
IRlum_F = pd.DataFrame(columns = ["id", "redshift", "IRLum_tot", "fAGN_IR","fAGN_err", "best_fAGN"])
IR_lum = []

z = df_F["best.universe.redshift"]
fAGN = df_F["bayes.agn.fracAGN"]
fAGN_err = df_F["bayes.agn.fracAGN_err"]
best_fAGN = df_F['best.agn.fracAGN']

for i in tqdm(df_F.index):
    
    sed = Table.read(cat_dirF + str(df_F["id"][i]) + "_best_model.fits", format="fits").to_pandas()
    w_spec = np.array(sed["wavelength"])
    
    three = np.where(3000 <= sed["wavelength"])[0][0]
    thir = np.where(30000 <= sed["wavelength"])[0][0]
    
    IR_lum.append(np.trapz(sed["L_lambda_total"][three:thir], w_spec[three:thir]))
    

IRlum_F = pd.concat([IRlum_F, pd.DataFrame(np.column_stack([df_F["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_F.columns)], ignore_index=True)

100%|██████████| 13136/13136 [13:23<00:00, 16.36it/s]
/var/lib/condor/execute/dir_335412/ipykernel_335502/1765798530.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IRlum_F = pd.concat([IRlum_F, pd.DataFrame(np.column_stack([df_F["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_F.columns)], ignore_index=True)


In [20]:
IRlum_COSMOSF = pd.DataFrame(columns = ["id", "redshift", "IRLum_tot", "fAGN_IR","fAGN_err", 'best_fAGN'])
IR_lum = []

z = df_COSMOSF["best.universe.redshift"]
fAGN = df_COSMOSF["bayes.agn.fracAGN"]
fAGN_err = df_COSMOSF["bayes.agn.fracAGN_err"]
best_fAGN = df_COSMOSF["best.agn.fracAGN"]

for i in tqdm(df_COSMOSF.index):
    
    sed = Table.read(COSMOS_dirF + str(df_COSMOSF["id"][i]) + "_best_model.fits", format="fits").to_pandas()
    w_spec = np.array(sed["wavelength"])
    
    three = np.where(3000 <= sed["wavelength"])[0][0]
    thir = np.where(30000 <= sed["wavelength"])[0][0]
    
    IR_lum.append(np.trapz(sed["L_lambda_total"][three:thir], w_spec[three:thir]))
    

IRlum_COSMOSF = pd.concat([IRlum_COSMOSF, pd.DataFrame(np.column_stack([df_COSMOSF["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_COSMOSF.columns)], ignore_index=True)

100%|██████████| 41766/41766 [32:02<00:00, 21.72it/s]
/var/lib/condor/execute/dir_335412/ipykernel_335502/2189922157.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IRlum_COSMOSF = pd.concat([IRlum_COSMOSF, pd.DataFrame(np.column_stack([df_COSMOSF["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_COSMOSF.columns)], ignore_index=True)


In [21]:
IRlum_COSMOSSK = pd.DataFrame(columns = ["id", "redshift", "IRLum_tot", "fAGN_IR","fAGN_err", "best_fAGN"])
IR_lum = []

z = df_COSMOSSK["best.universe.redshift"]
fAGN = df_COSMOSSK["bayes.agn.fracAGN"]
fAGN_err = df_COSMOSSK["bayes.agn.fracAGN_err"]
best_fAGN = df_COSMOSSK["best.agn.fracAGN"]

for i in tqdm(df_COSMOSSK.index):
    
    sed = Table.read(COSMOS_dirSK + str(df_COSMOSSK["id"][i]) + "_best_model.fits", format="fits").to_pandas()
    w_spec = np.array(sed["wavelength"])
    
    three = np.where(3000 <= sed["wavelength"])[0][0]
    thir = np.where(30000 <= sed["wavelength"])[0][0]
    
    IR_lum.append(np.trapz(sed["L_lambda_total"][three:thir], w_spec[three:thir]))
    

IRlum_COSMOSSK = pd.concat([IRlum_COSMOSSK, pd.DataFrame(np.column_stack([df_COSMOSSK["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_COSMOSSK.columns)], ignore_index=True)

100%|██████████| 31829/31829 [23:33<00:00, 22.52it/s]
/var/lib/condor/execute/dir_335412/ipykernel_335502/3326057741.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IRlum_COSMOSSK = pd.concat([IRlum_COSMOSSK, pd.DataFrame(np.column_stack([df_COSMOSSK["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_COSMOSSK.columns)], ignore_index=True)


In [22]:
#Save all tables

IRlum_SK.to_csv('IRlum_3-30SK.csv', index =False)
IRlum_F.to_csv('IRlum_3-30F.csv', index = False)
# IRlum_ANT.to_csv('IRlum_ANT.csv', index = False)
IRlum_COSMOSF.to_csv('IRlum_3-30F_COSMOS.csv', index = False)
IRlum_COSMOSSK.to_csv('IRlum_3-30SK_COSMOS.csv', index = False)

# 8-1000

## Paths

In [25]:
cat_dirSK = "/net/virgo01/data/users/ebbers/BSc_Thesis/SKIRTOR/8-1000SK/"
cat_dirF = "/net/virgo01/data/users/ebbers/BSc_Thesis/Fritz/8-1000F/"

COSMOS_dirSK = "/net/virgo01/data/users/ebbers/BSc_Thesis/COSMOS/SKIRTOR/SK8-1000/"
COSMOS_dirF= "/net/virgo01/data/users/ebbers/BSc_Thesis/COSMOS/Fritz/F8-1000/"

sed_dirSK = cat_dirSK
sed_dirF = cat_dirF

cat_nameSK = "results_8-1000SK.fits"
cat_nameF = "results_8-1000F.fits"

COSMOS_nameSK = "results_SK8-1000.fits"
COSMOS_nameF = 'results_F8-1000_COSMOS.fits'

observations_fileSK = cat_dirSK + "observations_8-1000SK.fits"
observations_fileF = cat_dirF + "observations_8-1000F.fits"
COSMOS_observations_fileSK = COSMOS_dirSK + 'observations_SK8-1000.fits'
COSMOS_observations_fileF = COSMOS_dirF + 'observations_F8-1000_COSMOS.fits'

resultsSK = Table.read(cat_dirSK + cat_nameSK, format="fits").to_pandas()
resultsF = Table.read(cat_dirF + cat_nameF, format="fits").to_pandas()
COSMOS_resultsSK = Table.read(COSMOS_dirSK + COSMOS_nameSK, format="fits").to_pandas()
COSMOS_resultsF = Table.read(COSMOS_dirF + COSMOS_nameF, format="fits").to_pandas()

In [26]:
observationsSK = Table.read(observations_fileSK, format="fits").to_pandas()

dfSK = pd.merge(
    resultsSK[useful_cols],
    observationsSK,
    on="id",
)

observationsF = Table.read(observations_fileF, format="fits").to_pandas()

dfF = pd.merge(
    resultsF[useful_cols],
    observationsF,
    on="id",
)

COSMOS_observationsSK = Table.read(COSMOS_observations_fileSK, format="fits").to_pandas()

dfCOSMOSSK = pd.merge(
    COSMOS_resultsSK[useful_cols],
    COSMOS_observationsSK,
    on="id",
)

COSMOS_observationsF = Table.read(COSMOS_observations_fileF, format="fits").to_pandas()

dfCOSMOSF = pd.merge(
    COSMOS_resultsF[useful_cols],
    COSMOS_observationsF,
    on="id",
)

In [27]:
mass_ratio = dfSK["best.stellar.m_star"] / dfSK["bayes.stellar.m_star"]
sfr_ratio = dfSK["best.sfh.sfr"] / dfSK["bayes.sfh.sfr"]
fagn_ratio = dfSK["best.agn.fracAGN"] / dfSK["bayes.agn.fracAGN"]
SN_ratio = dfSK["bayes.agn.fracAGN"]/ dfSK["bayes.agn.fracAGN_err"]

mass_sel = (mass_ratio >= 1 / 5) & (mass_ratio <= 5)
sfr_sel = (sfr_ratio >= 1 / 5) & (sfr_ratio <= 5)
fagn_sel = (fagn_ratio >= 1 / 5) & (fagn_ratio <= 5)
SN_sel = (SN_ratio > 1)

chi_sel = dfSK["best.reduced_chi_square"] < 5
selection = mass_sel & sfr_sel & fagn_sel & chi_sel & SN_sel

df_SK = dfSK[selection]
df_SK = df_SK.sort_values("id")

df_SK

,id,best.universe.redshift,bayes.agn.fracAGN,best.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.luminosity,best.agn.luminosity,bayes.agn.total_dust_luminosity,best.agn.total_dust_luminosity,bayes.dust.luminosity,...,herschel.pacs.160,herschel.pacs.160_err,PSW,PSW_err,PMW,PMW_err,PLW,PLW_err,eROSITA_soft,eROSITA_soft_err
388,39890303330569440,0.590000,0.361244,0.40,0.126204,8.842261e+37,1.071362e+38,8.799576e+37,1.066778e+38,1.396355e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19926,39890307625535289,0.690000,0.342935,0.40,0.093263,1.138522e+38,1.329247e+38,1.133432e+38,1.323560e+38,1.854646e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1840,39890316215468063,0.560000,0.241317,0.70,0.217114,2.367119e+37,7.521446e+37,2.351846e+37,7.489262e+37,6.363626e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6300,39890316215471241,0.509000,0.310107,0.30,0.206466,2.104557e+37,1.212425e+37,2.082524e+37,1.207237e+37,3.176667e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000002,6.698264e-07
15634,39890445064488671,0.620000,0.296239,0.55,0.170980,3.684815e+37,7.083900e+37,3.657763e+37,7.053588e+37,7.454587e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26143,43132745941012911,0.650000,0.394977,0.30,0.142945,1.212489e+38,9.164586e+37,1.197028e+38,8.982919e+37,1.613712e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4019,43132745941017875,0.745100,0.125470,0.10,0.098943,7.787204e+37,9.295626e+37,7.700201e+37,9.255851e+37,4.522465e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000002,6.739018e-07
27069,43132758825915994,0.340000,0.126028,0.10,0.099282,9.259293e+36,8.476964e+36,9.193920e+36,8.407494e+36,6.240490e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10126,43132771710821413,0.595074,0.336841,0.40,0.200944,5.331384e+38,1.158925e+39,5.248263e+38,1.149428e+39,7.858937e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000008,1.413871e-06


In [28]:
mass_ratio = dfF["best.stellar.m_star"] / dfF["bayes.stellar.m_star"]
sfr_ratio = dfF["best.sfh.sfr"] / dfF["bayes.sfh.sfr"]
fagn_ratio = dfF["best.agn.fracAGN"] / dfF["bayes.agn.fracAGN"]
SN_ratio = dfF["bayes.agn.fracAGN"]/ dfF["bayes.agn.fracAGN_err"]

mass_sel = (mass_ratio >= 1 / 5) & (mass_ratio <= 5)
sfr_sel = (sfr_ratio >= 1 / 5) & (sfr_ratio <= 5)
fagn_sel = (fagn_ratio >= 1 / 5) & (fagn_ratio <= 5)
SN_sel = (SN_ratio > 1)

chi_sel = dfF["best.reduced_chi_square"] < 5
selection = mass_sel & sfr_sel & fagn_sel & chi_sel & SN_sel

df_F = dfF[selection]

df_F = df_F.sort_values("id")
df_F

,id,best.universe.redshift,bayes.agn.fracAGN,best.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.luminosity,best.agn.luminosity,bayes.agn.total_dust_luminosity,best.agn.total_dust_luminosity,bayes.dust.luminosity,...,herschel.pacs.160,herschel.pacs.160_err,PSW,PSW_err,PMW,PMW_err,PLW,PLW_err,eROSITA_soft,eROSITA_soft_err
388,39890303330569440,0.590000,0.602024,0.40,0.155953,2.809729e+38,1.774363e+38,2.758101e+38,1.730467e+38,1.566264e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19926,39890307625535289,0.690000,0.775479,0.85,0.119693,7.939750e+38,1.005351e+39,7.852953e+38,9.962268e+38,1.766475e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1840,39890316215468063,0.560000,0.374928,0.55,0.263393,5.880473e+37,4.561954e+37,5.752842e+37,4.321143e+37,6.476313e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13465,39890316215470071,0.490000,0.356477,0.70,0.298986,7.192769e+37,1.611294e+38,7.118655e+37,1.596670e+38,6.096185e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6300,39890316215471241,0.509000,0.262922,0.20,0.178040,1.435303e+37,6.232745e+36,1.388531e+37,6.176177e+36,3.258404e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000002,6.698264e-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41499,43132771710818679,0.580000,0.267053,0.55,0.229570,7.742595e+37,1.769183e+38,7.649387e+37,1.753126e+38,1.327971e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10126,43132771710821413,0.595074,0.267059,0.30,0.184656,2.984816e+38,8.102984e+38,2.874765e+38,7.675254e+38,7.933739e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000008,1.413871e-06
42109,43132771710826283,0.580000,0.385129,0.70,0.244975,1.172572e+38,3.137105e+38,1.156116e+38,3.108632e+38,1.340968e+38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27217,43132776005782281,0.290000,0.411390,0.55,0.275887,1.912536e+37,1.558223e+37,1.886161e+37,1.519674e+37,1.497392e+37,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
mass_ratio = dfCOSMOSSK["best.stellar.m_star"] / dfCOSMOSSK["bayes.stellar.m_star"]
sfr_ratio = dfCOSMOSSK["best.sfh.sfr"] / dfCOSMOSSK["bayes.sfh.sfr"]
fagn_ratio = dfCOSMOSSK["best.agn.fracAGN"] / dfCOSMOSSK["bayes.agn.fracAGN"]
SN_ratio = dfCOSMOSSK["bayes.agn.fracAGN"]/ dfCOSMOSSK["bayes.agn.fracAGN_err"]

mass_sel = (mass_ratio >= 1 / 5) & (mass_ratio <= 5)
sfr_sel = (sfr_ratio >= 1 / 5) & (sfr_ratio <= 5)
fagn_sel = (fagn_ratio >= 1 / 5) & (fagn_ratio <= 5)
SN_sel = (SN_ratio > 1)

chi_sel = dfCOSMOSSK["best.reduced_chi_square"] < 5
selection = mass_sel & sfr_sel & fagn_sel & chi_sel & SN_sel

df_COSMOSSK = dfCOSMOSSK[selection]

df_COSMOSSK = df_COSMOSSK.sort_values("id")
df_COSMOSSK

,id,best.universe.redshift,bayes.agn.fracAGN,best.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.luminosity,best.agn.luminosity,bayes.agn.total_dust_luminosity,best.agn.total_dust_luminosity,bayes.dust.luminosity,...,subaru.suprime.IA767,subaru.suprime.IA767_err,subaru.suprime.IB827,subaru.suprime.IB827_err,subaru.suprime.NB711,subaru.suprime.NB711_err,subaru.suprime.NB816,subaru.suprime.NB816_err,vista.vircam.NB118,vista.vircam.NB118_err
62845,274.0,2.1539,0.303292,0.40,0.094520,4.971844e+37,4.777984e+37,3.139569e+37,4.683272e+37,4.899451e+37,...,0.001209,0.000123,0.001177,0.000121,0.001395,0.000142,0.001070,0.000108,0.001810,0.000187
25955,670.0,1.1792,0.435028,0.40,0.102657,1.942275e+37,1.964567e+37,6.362243e+36,5.950924e+36,3.668468e+37,...,0.002830,0.000285,0.004133,0.000415,0.002753,0.000278,0.004291,0.000430,0.005639,0.000566
62847,947.0,2.0474,0.457221,0.40,0.121439,2.753842e+38,2.676765e+38,2.741987e+38,2.665312e+38,2.942533e+38,...,0.000253,0.000038,0.000442,0.000056,0.000121,0.000039,0.000312,0.000036,0.002052,0.000211
47968,1068.0,1.6465,0.795748,0.85,0.083244,3.292150e+37,3.488645e+37,3.261282e+37,3.460055e+37,6.316587e+36,...,0.000098,0.000024,0.000101,0.000026,0.000133,0.000030,0.000107,0.000019,0.000295,0.000045
72689,1097.0,2.8639,0.737878,0.99,0.303276,1.231485e+38,2.428216e+38,1.211295e+38,2.408316e+38,2.962249e+36,...,0.000074,0.000026,0.000071,0.000029,0.000052,0.000025,0.000067,0.000016,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62843,964384.0,1.6049,0.482366,0.70,0.274070,4.575995e+36,4.322567e+36,4.284750e+36,4.236882e+36,3.062309e+36,...,0.000283,0.000034,0.000294,0.000035,0.000243,0.000031,0.000326,0.000036,NaN,NaN
47964,964386.0,1.3068,0.589256,0.70,0.208200,3.441849e+36,2.781638e+36,3.221754e+36,2.726498e+36,6.335196e+36,...,0.000255,0.000032,0.000196,0.000028,0.000288,0.000036,0.000266,0.000030,NaN,NaN
80474,964391.0,2.8983,0.306818,0.10,0.296425,2.085795e+37,3.655800e+36,2.047071e+37,1.281415e+36,1.063148e+37,...,0.000006,0.000019,0.000030,0.000020,0.000075,0.000021,0.000040,0.000015,NaN,NaN
47967,964428.0,1.2587,0.382373,0.55,0.157672,7.684803e+36,1.262061e+37,6.780734e+36,1.256661e+37,3.562242e+37,...,0.002323,0.000237,0.002577,0.000259,NaN,NaN,NaN,NaN,0.007177,0.000719


In [30]:
mass_ratio = dfCOSMOSF["best.stellar.m_star"] / dfCOSMOSF["bayes.stellar.m_star"]
sfr_ratio = dfCOSMOSF["best.sfh.sfr"] / dfCOSMOSF["bayes.sfh.sfr"]
fagn_ratio = dfCOSMOSF["best.agn.fracAGN"] / dfCOSMOSF["bayes.agn.fracAGN"]
SN_ratio = dfCOSMOSF["bayes.agn.fracAGN"]/ dfCOSMOSF["bayes.agn.fracAGN_err"]

mass_sel = (mass_ratio >= 1 / 5) & (mass_ratio <= 5)
sfr_sel = (sfr_ratio >= 1 / 5) & (sfr_ratio <= 5)
fagn_sel = (fagn_ratio >= 1 / 5) & (fagn_ratio <= 5)
SN_sel = (SN_ratio > 1)

chi_sel = dfCOSMOSF["best.reduced_chi_square"] < 5
selection = mass_sel & sfr_sel & fagn_sel & chi_sel & SN_sel

df_COSMOSF = dfCOSMOSF[selection]

df_COSMOSF = df_COSMOSF.sort_values("id")
df_COSMOSF

,id,best.universe.redshift,bayes.agn.fracAGN,best.agn.fracAGN,bayes.agn.fracAGN_err,bayes.agn.luminosity,best.agn.luminosity,bayes.agn.total_dust_luminosity,best.agn.total_dust_luminosity,bayes.dust.luminosity,...,subaru.suprime.IA767,subaru.suprime.IA767_err,subaru.suprime.IB827,subaru.suprime.IB827_err,subaru.suprime.NB711,subaru.suprime.NB711_err,subaru.suprime.NB816,subaru.suprime.NB816_err,vista.vircam.NB118,vista.vircam.NB118_err
62845,274.0,2.1539,0.172665,0.20,0.079866,4.731353e+37,4.212636e+37,1.345881e+37,1.335085e+37,3.920566e+37,...,0.001209,0.000123,0.001177,0.000121,0.001395,0.000142,0.001070,0.000108,0.001810,0.000187
25955,670.0,1.1792,0.144004,0.10,0.093663,2.212935e+37,1.787657e+37,7.535238e+36,4.955933e+36,2.433227e+37,...,0.002830,0.000285,0.004133,0.000415,0.002753,0.000278,0.004291,0.000430,0.005639,0.000566
25957,707.0,1.1951,0.354749,0.10,0.340908,1.262337e+37,3.834771e+35,1.240808e+37,9.806517e+34,1.339660e+36,...,0.000110,0.000027,0.000272,0.000041,0.000078,0.000028,0.000218,0.000026,0.000177,0.000044
12781,942.0,0.9000,0.330497,0.70,0.302387,9.542876e+36,1.030113e+37,9.419398e+36,1.004629e+37,4.873882e+36,...,0.000737,0.000078,0.000860,0.000090,0.000737,0.000080,0.000978,0.000101,0.001403,0.000146
62847,947.0,2.0474,0.529115,0.55,0.057206,4.384147e+38,4.793148e+38,4.274240e+38,4.674572e+38,3.436397e+38,...,0.000253,0.000038,0.000442,0.000056,0.000121,0.000039,0.000312,0.000036,0.002052,0.000211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62843,964384.0,1.6049,0.533664,0.99,0.342450,4.928287e+37,4.076448e+36,4.831751e+37,1.291923e+36,3.154234e+36,...,0.000283,0.000034,0.000294,0.000035,0.000243,0.000031,0.000326,0.000036,NaN,NaN
72688,964385.0,2.4158,0.492334,0.99,0.376541,4.027921e+38,2.410626e+39,3.847749e+38,2.283377e+39,1.242378e+37,...,0.000111,0.000022,0.000092,0.000022,0.000031,0.000020,0.000119,0.000019,NaN,NaN
47964,964386.0,1.3068,0.417324,0.70,0.253637,1.144292e+37,2.202490e+37,1.099490e+37,2.148003e+37,7.091153e+36,...,0.000255,0.000032,0.000196,0.000028,0.000288,0.000036,0.000266,0.000030,NaN,NaN
47967,964428.0,1.2587,0.550630,0.85,0.266984,9.128600e+37,1.882543e+38,9.022458e+37,1.865457e+38,3.635376e+37,...,0.002323,0.000237,0.002577,0.000259,NaN,NaN,NaN,NaN,0.007177,0.000719


In [31]:
IRlum_SK = pd.DataFrame(columns = ["id", "redshift", "IRLum_tot", "fAGN_IR","fAGN_err", "best_fAGN"])

z = df_SK["best.universe.redshift"]
fAGN = df_SK["bayes.agn.fracAGN"]
fAGN_err = df_SK["bayes.agn.fracAGN_err"]
best_fAGN = df_SK["best.agn.fracAGN"]

IR_lum = []

for i in tqdm(df_SK.index):
    
    sed = Table.read(cat_dirSK + str(df_SK["id"][i]) + "_best_model.fits", format="fits").to_pandas()

    w_spec = np.array(sed["wavelength"])
    
    eight = np.where(8000 <= sed["wavelength"])[0][0]
    thous = np.where(10**6 <= sed["wavelength"])[0][0]
    
    IR_lum.append(np.trapz(sed["L_lambda_total"][eight:thous], w_spec[eight:thous]))
    

IRlum_SK = pd.concat([IRlum_SK, pd.DataFrame(np.column_stack([df_SK["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_SK.columns)], ignore_index=True)

100%|██████████| 4897/4897 [05:11<00:00, 15.72it/s]
/var/lib/condor/execute/dir_335412/ipykernel_335502/3431022009.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IRlum_SK = pd.concat([IRlum_SK, pd.DataFrame(np.column_stack([df_SK["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_SK.columns)], ignore_index=True)


In [32]:
IRlum_F = pd.DataFrame(columns = ["id", "redshift", "IRLum_tot", "fAGN_IR","fAGN_err", "best_fAGN"])
IR_lum = []

z = df_F["best.universe.redshift"]
fAGN = df_F["bayes.agn.fracAGN"]
fAGN_err = df_F["bayes.agn.fracAGN_err"]
best_fAGN = df_F["best.agn.fracAGN"]

for i in tqdm(df_F.index):
    
    sed = Table.read(cat_dirF + str(df_F["id"][i]) + "_best_model.fits", format="fits").to_pandas()
    w_spec = np.array(sed["wavelength"])
    
    eight = np.where(8000 <= sed["wavelength"])[0][0]
    thous = np.where(10**6 <= sed["wavelength"])[0][0]
    
    IR_lum.append(np.trapz(sed["L_lambda_total"][eight:thous], w_spec[eight:thous]))
    

IRlum_F = pd.concat([IRlum_F, pd.DataFrame(np.column_stack([df_F["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_F.columns)], ignore_index=True)

100%|██████████| 10923/10923 [08:16<00:00, 22.02it/s]
/var/lib/condor/execute/dir_335412/ipykernel_335502/1896949496.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IRlum_F = pd.concat([IRlum_F, pd.DataFrame(np.column_stack([df_F["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_F.columns)], ignore_index=True)


In [33]:
IRlum_COSMOS_SK = pd.DataFrame(columns = ["id", "redshift", "IRLum_tot", "fAGN_IR","fAGN_err", "best_fAGN"])

z = df_COSMOSSK["best.universe.redshift"]
fAGN = df_COSMOSSK["bayes.agn.fracAGN"]
fAGN_err = df_COSMOSSK["bayes.agn.fracAGN_err"]
best_fAGN = df_COSMOSSK["best.agn.fracAGN"]

IR_lum = []

for i in tqdm(df_COSMOSSK.index):
    
    sed = Table.read(COSMOS_dirSK + str(df_COSMOSSK["id"][i]) + "_best_model.fits", format="fits").to_pandas()

    w_spec = np.array(sed["wavelength"])
    
    eight = np.where(8000 <= sed["wavelength"])[0][0]
    thous = np.where(10**6 <= sed["wavelength"])[0][0]
    
    IR_lum.append(np.trapz(sed["L_lambda_total"][eight:thous], w_spec[eight:thous]))
    

IRlum_COSMOS_SK = pd.concat([IRlum_COSMOS_SK, pd.DataFrame(np.column_stack([df_COSMOSSK["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_COSMOS_SK.columns)], ignore_index=True)

100%|██████████| 28829/28829 [22:59<00:00, 20.89it/s]
/var/lib/condor/execute/dir_335412/ipykernel_335502/4110100547.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IRlum_COSMOS_SK = pd.concat([IRlum_COSMOS_SK, pd.DataFrame(np.column_stack([df_COSMOSSK["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_COSMOS_SK.columns)], ignore_index=True)


In [35]:
IRlum_COSMOS8F = pd.DataFrame(columns = ["id", "redshift", "IRLum_tot", "fAGN_IR","fAGN_err", "best_fAGN"])
IR_lum = []

z = df_COSMOSF["best.universe.redshift"]
fAGN = df_COSMOSF["bayes.agn.fracAGN"]
fAGN_err = df_COSMOSF["bayes.agn.fracAGN_err"]
best_fAGN = df_COSMOSF["best.agn.fracAGN"]

for i in tqdm(df_COSMOSF.index):
    
    sed = Table.read(COSMOS_dirF + str(df_COSMOSF["id"][i]) + "_best_model.fits", format="fits").to_pandas()
    w_spec = np.array(sed["wavelength"])
    
    eight = np.where(8000 <= sed["wavelength"])[0][0]
    thous = np.where(10**6 <= sed["wavelength"])[0][0]
    
    IR_lum.append(np.trapz(sed["L_lambda_total"][eight:thous], w_spec[eight:thous]))
    

IRlum_COSMOS8F = pd.concat([IRlum_COSMOS8F, pd.DataFrame(np.column_stack([df_COSMOSF["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_COSMOS8F.columns)], ignore_index=True)

100%|██████████| 37854/37854 [09:11<00:00, 68.60it/s]
/var/lib/condor/execute/dir_335412/ipykernel_335502/1967464438.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  IRlum_COSMOS8F = pd.concat([IRlum_COSMOS8F, pd.DataFrame(np.column_stack([df_COSMOSF["id"], z, IR_lum, fAGN, fAGN_err, best_fAGN]), columns=IRlum_COSMOS8F.columns)], ignore_index=True)


In [36]:
#Save both tables

IRlum_SK.to_csv('IRlum_8-1000SK.csv', index=False)
IRlum_F.to_csv('IRlum_8-1000F.csv', index = False)
IRlum_COSMOS_SK.to_csv('IRlum_8-1000SK_COSMOS.csv', index = False)
IRlum_COSMOS8F.to_csv('IRlum_8-1000F_COSMOS.csv', index = False)

In [19]:
#Check to see if integration in DataSaver file is correct

IRlum_COSMOSF = pd.read_csv('IRlum_8-1000F_COSMOS.csv')

IRlum_COSMOSF

print(df_COSMOSF[df_COSMOSF["id"].isin([942.0])]["best.dust.luminosity"])

print(IRlum_COSMOSF[IRlum_COSMOSF["id"].isin([float(942.0)])]["IRLum_tot"])
print(df_COSMOSF[df_COSMOSF["id"].isin([float(942.0)])]["best.agn.fracAGN"])

12781    3.963077e+36
Name: best.dust.luminosity, dtype: float64
3    1.415322e+37
Name: IRLum_tot, dtype: float64
12781    0.7
Name: best.agn.fracAGN, dtype: float64
